In [32]:

import pandas as pd
games = pd.read_csv('games.csv')
plays = pd.read_csv('plays.csv')
print(plays.shape[0])

12486


In [33]:


team_dict = {'Team': [], 'pass_yardage' : [], 'total_pass_plays' : [], 'total_run_plays': [], 
             'run_yardage': [], 'total_yardage': [], 'pct_run' : [], 'total_plays' : [], 'penalty_yards' : [], 'points' : [], 'wins': []}

#First, populate the teams using games.csv: 

for i in range(0, games.shape[0]): 
    if games['homeTeamAbbr'][i] not in team_dict['Team']: 
        team_dict['Team'].append(games['homeTeamAbbr'][i])
    if games['visitorTeamAbbr'][i] not in team_dict['Team']: 
        team_dict['Team'].append(games['visitorTeamAbbr'][i])

## Map to quickly sort through team_dict and add easier 
team_map = dict()
for i in range(len(team_dict['Team'])): 
    team_map[team_dict['Team'][i]] =  i
    

In [34]:
## zero-initialize all the arrays 
for i in range(32): 
    team_dict['pass_yardage'].append(0)
    team_dict['run_yardage'].append(0)
    team_dict['total_yardage'].append(0)
    team_dict['total_pass_plays'].append(0)
    team_dict['total_run_plays'].append(0)
    team_dict['penalty_yards'].append(0)
    team_dict['points'].append(0)
    team_dict['wins'].append(0)

In [35]:
#iterate and add yardage/plays 

for i in range(0, plays.shape[0]): 
    team = plays['possessionTeam'][i]
    x = team_map[team]
    penalty_yards = plays['playResult'][i] - plays['prePenaltyPlayResult'][i]
    team_dict['penalty_yards'][x] += penalty_yards
    if plays['passResult'][i] == "C": 
        team_dict['total_pass_plays'][x] += 1 
        yardage = int(plays['prePenaltyPlayResult'][i])
        team_dict['total_yardage'][x] += yardage
        team_dict['pass_yardage'][x] += yardage
## Categorizing pass plays that scramble and turn into runs as runs, can change 
    else: 
        team_dict['total_run_plays'][x] += 1 
        yardage = int(plays['prePenaltyPlayResult'][i])
        team_dict['total_yardage'][x] += yardage
        team_dict['run_yardage'][x] += yardage


#Adding in points scored from games.csv 
for i in range(0, games.shape[0]):
    team_dict['points'][team_map[games['homeTeamAbbr'][i]]] += games['homeFinalScore'][i]
    team_dict['points'][team_map[games['visitorTeamAbbr'][i]]] += games['visitorFinalScore'][i]
    if games['homeFinalScore'][i] > games['visitorFinalScore'][i]: 
        team_dict['wins'][team_map[games['homeTeamAbbr'][i]]] += 1
    elif games['visitorFinalScore'][i] > games['homeFinalScore'][i]: 
        team_dict['wins'][team_map[games['visitorTeamAbbr'][i]]] += 1

In [36]:
#Calculate pct_run 
for i in range(32): 
    total_plays = team_dict['total_run_plays'][i] + team_dict['total_pass_plays'][i]
    if total_plays == 0: \
        team_dict['pct_run'].append(0)
    else: 
        team_dict['pct_run'].append( team_dict['total_run_plays'][i]  / total_plays)


In [37]:

#Calculate total plays 
for i in range(32): 
    team_dict['total_plays'].append(team_dict['total_pass_plays'][i] + team_dict['total_run_plays'][i])

In [38]:
team_df = pd.DataFrame(team_dict)
team_df.to_csv('stat202.csv')